In [770]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
 
from sklearn import metrics #метрики
from sklearn import model_selection #методы разделения и валидации
from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
# plt.style.use('seaborn') #стиль отрисовки seaborn
%matplotlib inline

# для нормализации, стандартизации
from sklearn import preprocessing

import category_encoders as ce # импорт для работы с кодировщиком

In [771]:
df = pd.read_csv('D:\\main\\1_projects\educational_projects\\project_kaggle_car_price\\data\\train_data.csv')
df.head()

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0
3,3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,NaN,1,For Sale,14488.0
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0


In [772]:
df.shape

(35000, 15)

In [773]:
df.isnull().mean() * 100

row_ID                     0.000000
vehicle_manufacturer       0.002857
vehicle_model              0.020000
vehicle_category           0.002857
current_mileage            0.000000
vehicle_year               0.000000
vehicle_gearbox_type       0.002857
doors_cnt                  2.128571
wheels                     0.002857
vehicle_color              1.145714
vehicle_interior_color    19.194286
car_vin                   67.177143
car_leather_interior       0.000000
deal_type                  0.000000
final_price                0.000000
dtype: float64

In [774]:
#создаем копию исходной таблицы
drop_data = df.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
drop_data = drop_data.dropna(how='any', axis=0)
drop_data = drop_data.drop('row_ID', axis=1)
#отображаем результирующую долю пропусков
print(drop_data.isnull().mean())
print(drop_data.shape)


vehicle_manufacturer      0.0
vehicle_model             0.0
vehicle_category          0.0
current_mileage           0.0
vehicle_year              0.0
vehicle_gearbox_type      0.0
doors_cnt                 0.0
wheels                    0.0
vehicle_color             0.0
vehicle_interior_color    0.0
car_leather_interior      0.0
deal_type                 0.0
final_price               0.0
dtype: float64
(28118, 13)


In [775]:
print(drop_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28118 entries, 0 to 34999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   vehicle_manufacturer    28118 non-null  object 
 1   vehicle_model           28118 non-null  object 
 2   vehicle_category        28118 non-null  object 
 3   current_mileage         28118 non-null  int64  
 4   vehicle_year            28118 non-null  int64  
 5   vehicle_gearbox_type    28118 non-null  object 
 6   doors_cnt               28118 non-null  object 
 7   wheels                  28118 non-null  object 
 8   vehicle_color           28118 non-null  object 
 9   vehicle_interior_color  28118 non-null  object 
 10  car_leather_interior    28118 non-null  int64  
 11  deal_type               28118 non-null  object 
 12  final_price             28118 non-null  float64
dtypes: float64(1), int64(3), object(9)
memory usage: 3.0+ MB
None


In [776]:
X = drop_data.drop('final_price', axis=1).select_dtypes(exclude='object')
y = drop_data['final_price']
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

Train shape: (22494, 3)
Valid shape: (5624, 3)


Используем модель линейной регрессии

In [777]:
# обучаем модель
model = linear_model.LinearRegression(
    
).fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

MAE score: 4441.860 thou. $
RMSE score: 72378.851 thou. $
MAPE score: 192685149920993536.000 %
R2 score: 0.001


In [778]:
# обучаем модель
model = linear_model.SGDRegressor(
    
).fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

MAE score: 168978657392520543076352.000 thou. $
RMSE score: 4355513450909304434458624.000 thou. $
MAPE score: 261252017411516370909203952413179904.000 %
R2 score: -3619370426897763759862388822844079341568.000


## Итерация 2

небольная обработка

In [779]:
drop_data['deal_type'] = drop_data['deal_type'].apply(lambda x: 'sale' if 'Sale' in x else 'rent')
drop_data['vehicle_age'] = 2020 - drop_data['vehicle_year']

print(drop_data.shape)

(28118, 14)


In [780]:
X = drop_data.drop('final_price', axis=1).select_dtypes(exclude='object')
y = drop_data['final_price']
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

# обучаем модель
model = linear_model.LinearRegression().fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

Train shape: (22494, 4)
Valid shape: (5624, 4)
MAE score: 4441.860 thou. $
RMSE score: 72378.851 thou. $
MAPE score: 192685149920994112.000 %
R2 score: 0.001


### Итерация 3

Пробуем оставить только авто на продажу

In [781]:
sale_data = drop_data[drop_data['deal_type'] == 'sale']


X = sale_data.drop('final_price', axis=1).select_dtypes(exclude='object')
y = sale_data['final_price']
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

# обучаем модель
model = linear_model.LinearRegression().fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

Train shape: (21992, 4)
Valid shape: (5498, 4)
MAE score: 5246.319 thou. $
RMSE score: 92488.650 thou. $
MAPE score: 131475557788164672.000 %
R2 score: 0.000


### Итерация 4

In [782]:
sale_data.head()

,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_leather_interior,deal_type,final_price,vehicle_age
0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,0,sale,3650.0,6
1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,0,sale,6800.0,10
2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,1,sale,6300.0,4
3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,1,sale,14488.0,5
5,MITSUBISHI,Delica,Jeep,149000,2003,Automatic,4/5,Right-hand drive,Silver,Black,0,sale,20.0,17


грубо избавимся от выбросов

In [783]:
def outliers_z_score(data, feature, log_scale=False, lower_b=1, upper_b=1):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = (mu - 3 * sigma)*lower_b
    upper_bound = (mu + 3 * sigma)*upper_b
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [784]:
outliers, cleaned = outliers_z_score(sale_data, 'current_mileage', log_scale=True, lower_b=0)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

sale_data = sale_data.drop(outliers.index, axis=0)

outliers, cleaned = outliers_z_score(sale_data, 'vehicle_age', log_scale=True, lower_b=0)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

sale_data = sale_data.drop(outliers.index, axis=0)

outliers, cleaned = outliers_z_score(sale_data, 'final_price', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

sale_data = sale_data.drop(outliers.index, axis=0)

Число выбросов по методу z-отклонения: 45
Результирующее число записей: 26432
Число выбросов по методу z-отклонения: 31
Результирующее число записей: 27336
Число выбросов по методу z-отклонения: 418
Результирующее число записей: 26996


In [785]:
X = sale_data.drop('final_price', axis=1).select_dtypes(exclude='object')
y = sale_data['final_price']
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

# обучаем модель
model = linear_model.LinearRegression().fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

Train shape: (21596, 4)
Valid shape: (5400, 4)
MAE score: 3327.997 thou. $
RMSE score: 5308.481 thou. $
MAPE score: 326.677 %
R2 score: 0.168


### Итерация 4

In [786]:
sale_data.head()

,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_leather_interior,deal_type,final_price,vehicle_age
0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,0,sale,3650.0,6
1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,0,sale,6800.0,10
2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,1,sale,6300.0,4
3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,1,sale,14488.0,5
6,HYUNDAI,Tucson,Jeep,173600,2013,Automatic,4/5,Left wheel,Silver,Black,1,sale,7632.0,7


In [787]:
top_vehicle_manufacturer = sale_data['vehicle_manufacturer'].value_counts().nlargest(20).index
sale_data['vehicle_manufacturer'] = sale_data['vehicle_manufacturer'].apply(lambda x: x if x in top_vehicle_manufacturer else 'other')

encoder = ce.OneHotEncoder(cols=['vehicle_manufacturer'], use_cat_names=True) # указываем столбец для кодирования
type_bin = encoder.fit_transform(sale_data['vehicle_manufacturer'])
sale_data = pd.concat([sale_data, type_bin], axis=1)

In [788]:
X = sale_data.drop('final_price', axis=1).select_dtypes(exclude='object')
y = sale_data['final_price']
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

# обучаем модель
model = linear_model.LinearRegression().fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

Train shape: (21596, 25)
Valid shape: (5400, 25)
MAE score: 3164.751 thou. $
RMSE score: 4982.706 thou. $
MAPE score: 277.668 %
R2 score: 0.267


In [597]:
X = sale_data.drop('final_price', axis=1).select_dtypes(exclude='object')
y = sale_data['final_price']
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

# обучаем модель
model = linear_model.LinearRegression().fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

ValueError: No axis named 1 for object type Series

определим число уникальных категорий в каждом столбце нашей таблицы

In [372]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in drop_data.select_dtypes(include='object'):
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, drop_data[col].nunique(),drop_data[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,wheels,2,object
1,deal_type,2,object
2,doors_cnt,3,object
3,vehicle_gearbox_type,4,object
4,vehicle_category,11,object
5,vehicle_interior_color,11,object
6,vehicle_color,16,object
7,vehicle_manufacturer,67,object
8,vehicle_model,2209,object


In [373]:
drop_data['wheels'] = drop_data['wheels'].apply(lambda x: 'left' if 'Left' in x else 'right')
drop_data['deal_type'] = drop_data['deal_type'].apply(lambda x: 'sale' if 'Sale' in x else 'rent')
drop_data['doors_cnt'] = drop_data['doors_cnt'].apply(lambda x: x.strip())
drop_data['vehicle_color'] = drop_data['vehicle_color'].apply(lambda x: x.strip())

pop_vehicle_color = drop_data['vehicle_color'].value_counts().nlargest(5).index
drop_data['vehicle_color'] = drop_data['vehicle_color'].apply(lambda x: x if x in pop_vehicle_color else 'other')

# pop_vehicle_manufacturer = drop_data['vehicle_manufacturer'].value_counts().nlargest(10).index
# drop_data['vehicle_manufacturer'] = drop_data['vehicle_manufacturer'].apply(lambda x: x if x in pop_vehicle_manufacturer else 'other')

drop_data['vehicle_age'] = 2020 - drop_data['vehicle_year']

drop_data.head(10)

,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_leather_interior,deal_type,final_price,vehicle_age
0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,right,Silver,Black,0,sale,3650.0,6
1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,left,Silver,Black,0,sale,6800.0,10
2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,left,Silver,Black,1,sale,6300.0,4
3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,left,Blue,Black,1,sale,14488.0,5
5,MITSUBISHI,Delica,Jeep,149000,2003,Automatic,4/5,right,Silver,Black,0,sale,20.0,17
6,HYUNDAI,Tucson,Jeep,173600,2013,Automatic,4/5,left,Silver,Black,1,sale,7632.0,7
7,BMW,X5,Jeep,264000,2006,Tiptronic,4/5,left,Silver,Black,1,sale,6500.0,14
8,TOYOTA,Aqua,Hatchback,86658,2017,Automatic,4/5,left,White,Black,1,sale,4870.0,3
9,BMW,640,Sedan,190000,2012,Tiptronic,4/5,left,Grey,Black,1,sale,16500.0,8
13,FORD,Focus,Sedan,116800,2013,Tiptronic,4/5,left,Black,Black,1,sale,4500.0,7


Ищем выбросы

In [374]:
# удаляем вручную выбросы по милям
mileage_outliers = drop_data[drop_data['current_mileage'] > 400000]
drop_data = drop_data.drop(mileage_outliers.index, axis=0)

# по возрасту
age_outliers = drop_data[drop_data['vehicle_age'] > 30]
drop_data = drop_data.drop(age_outliers.index, axis=0)

In [375]:
drop_data.shape

(27246, 14)

Кодируем признаки

In [376]:
bin_encoder = ce.BinaryEncoder(cols=['vehicle_manufacturer']) # указываем столбец для кодирования
type_bin = bin_encoder.fit_transform(drop_data['vehicle_manufacturer'])
drop_data = pd.concat([drop_data, type_bin], axis=1)

ord_encoder = ce.OrdinalEncoder()
data_bin = ord_encoder.fit_transform(drop_data[['vehicle_category']])
drop_data = pd.concat([drop_data, data_bin], axis=1)

data_bin = ord_encoder.fit_transform(drop_data[['vehicle_gearbox_type']])
drop_data = pd.concat([drop_data, data_bin], axis=1)

data_bin = ord_encoder.fit_transform(drop_data[['doors_cnt']])
drop_data = pd.concat([drop_data, data_bin], axis=1)

data_bin = ord_encoder.fit_transform(drop_data[['wheels']])
drop_data = pd.concat([drop_data, data_bin], axis=1)

drop_data = drop_data.select_dtypes(exclude='object')

In [377]:
drop_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27246 entries, 0 to 34999
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   current_mileage         27246 non-null  int64  
 1   vehicle_year            27246 non-null  int64  
 2   car_leather_interior    27246 non-null  int64  
 3   final_price             27246 non-null  float64
 4   vehicle_age             27246 non-null  int64  
 5   vehicle_manufacturer_0  27246 non-null  int64  
 6   vehicle_manufacturer_1  27246 non-null  int64  
 7   vehicle_manufacturer_2  27246 non-null  int64  
 8   vehicle_manufacturer_3  27246 non-null  int64  
 9   vehicle_manufacturer_4  27246 non-null  int64  
 10  vehicle_manufacturer_5  27246 non-null  int64  
 11  vehicle_manufacturer_6  27246 non-null  int64  
 12  vehicle_category        27246 non-null  int32  
 13  vehicle_gearbox_type    27246 non-null  int32  
 14  doors_cnt               27246 non-null

In [378]:
result_data = drop_data.drop(['vehicle_year', 'doors_cnt', 'wheels', 'vehicle_category', 'vehicle_gearbox_type'], axis=1)

In [379]:
X = result_data.drop('final_price', axis=1)
y = result_data['final_price']

In [380]:
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

# обучаем модель
model = linear_model.LinearRegression(
    
).fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)


print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

Train shape: (21796, 10)
Valid shape: (5450, 10)
MAE score: 4866.117 thou. $
RMSE score: 81157.600 thou. $
MAPE score: 145066029035940064.000 %
R2 score: 0.001
